In [1]:
import os
import glob

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

import sys
sys.path.append('..')

import modules.gait_metrics as gm
import modules.pose_estimation as pe


load_dir = os.path.join('..', 'data', 'kinect', 'best pos')
save_dir = os.path.join('..', 'data', 'results')

save_name = 'kinect_gait_metrics.csv'

# All files with .pkl extension
file_paths = glob.glob(os.path.join(load_dir, '*.pkl'))
save_path = os.path.join(save_dir, save_name)

df_metrics = pd.read_csv(save_path, index_col=0)


kinect_id = '2014-12-16_P005_Post_004'

matching_files = [f for f in file_paths if kinect_id in f]

for file_path in matching_files:

    df_head_feet = pd.read_pickle(file_path)
    
    # Convert all position vectors to float type 
    # so they can be easily input to linear algebra functions
    df_head_feet = df_head_feet.applymap(pd.to_numeric)

    # %% Peak detection

    # Cluster frames with k means to locate the 4 walking passes
    frames = df_head_feet.index.values.reshape(-1, 1)
    k_means = KMeans(n_clusters=4, random_state=0).fit(frames)

    foot_diff = df_head_feet.L_FOOT - df_head_feet.R_FOOT
    foot_dist = foot_diff.apply(np.linalg.norm)



In [2]:
frame_labels = k_means.labels_

pass_dfs = gm.split_by_pass(df_head_feet, frame_labels)

In [3]:
list_ = []

for i, df_pass in enumerate(pass_dfs):
    
    df_gait = gm.walking_pass_metrics(df_pass)
    
    # Add column to record the walking pass
    df_gait['pass'] = i

    list_.append(df_gait)
    
df_gait_total = pd.concat(list_)

# Reset the index because there are repeated index elements
df_gait_total = df_gait_total.reset_index(drop=True)

In [4]:
# Split DataFrame by side (right and left)
df_l, df_r = [x for _, x in df_gait_total.groupby('side')]

In [7]:
df = pd.merge(df_l, df_r, how='outer', left_on='pass', right_on='pass', suffixes=['_L', '_R'])

In [8]:
df.drop(list(df.filter(like='number')), axis=1)

,side_L,step_length_L,step_time_L,stride_width_L,stride_length_L,stride_time_L,stride_velocity_L,pass,side_R,step_length_R,step_time_R,stride_width_R,stride_length_R,stride_time_R,stride_velocity_R
0,L,51.396655,0.433333,12.042193,108.501030,0.966667,112.242445,1,R,57.433271,0.533333,10.360671,119.855167,1.000000,119.855167
1,L,53.601627,0.433333,6.535507,109.475407,0.833333,131.370488,3,R,55.789605,0.400000,7.218874,115.287294,0.966667,119.262718
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,R,61.887649,0.533333,8.541389,121.472407,0.933333,130.149008
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,R,68.164883,0.600000,6.095472,124.408967,1.000000,124.408967


In [9]:
strings_to_drop = ['side', 'pass', 'number']

In [10]:
import modules.pandas_funcs as pf

In [15]:
strings_to_drop = ['side', 'pass', 'number']

df = pf.drop_any_like(df, strings_to_drop, axis=1)

In [29]:
# Order column labels alphabetically
df = df.sort_index(axis=1)

In [32]:
df.mean()

step_length_L         52.499141
step_length_R         60.818852
step_time_L            0.433333
step_time_R            0.516667
stride_length_L      108.988219
stride_length_R      120.255959
stride_time_L          0.900000
stride_time_R          0.975000
stride_velocity_L    121.806467
stride_velocity_R    123.418965
stride_width_L         9.288850
stride_width_R         8.054102
dtype: float64